<a href="https://colab.research.google.com/github/amit-chaubey/fine-tune-models/blob/main/fine_tune_facebook_opt_350m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate rouge_score datasets transformers trl

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
# Load conversational dataset from Hugging Face
dataset = load_dataset("sweatSmile/marx-dataset", split="train")

# Load a small, efficient model suitable for fine-tuning
model_name = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define how conversational dataset should be formatted
def formatting_prompts_func(example):
    conversations = example['conversation']
    formatted_text = ""
    for conv in conversations:
        if conv["from"] == "human":
            formatted_text += f"### Human: {conv['value']}\n"
        else:
            formatted_text += f"### Assistant: {conv['value']}\n"
    return formatted_text

# Data collator to fine-tune on assistant responses only
instruction_template = "### Human:"
response_template = "### Assistant:"

collator = DataCollatorForCompletionOnlyLM(
    instruction_template=instruction_template,
    response_template=response_template,
    tokenizer=tokenizer,
    mlm=False
)

In [6]:
# Configure training arguments
training_args = SFTConfig(
    output_dir="./marx_finetuned_model",
    max_length=512,
    num_train_epochs=5,
    logging_steps=50,
    packing=False,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

# Start fine-tuning
trainer.train()


Repo card metadata block was not found. Setting CardData to empty.


Step,Training Loss
50,2.589100
100,2.479100
150,1.992300
200,1.643300
250,1.680700
300,1.157300
350,1.114700
400,0.943400
450,0.766100
500,0.784800


TrainOutput(global_step=625, training_loss=1.3268996353149414, metrics={'train_runtime': 154.8857, 'train_samples_per_second': 32.282, 'train_steps_per_second': 4.035, 'total_flos': 515014435405824.0, 'train_loss': 1.3268996353149414})

In [7]:
# Using the fine-tuned model for inference
model.eval()
input_text = "### Human: When was The Manifesto published?\n### Assistant:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Human: When was The Manifesto published?
### Assistant: The Manifesto was published in 1848.



In [11]:
# Additional Training Configuration Tweaks
training_args = SFTConfig(
    output_dir="./marx_finetuned_model_v2",
    max_length=512,
    num_train_epochs=10,  # Increased epochs
    learning_rate=3e-5,   # Custom learning rate
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=25,
    save_steps=100,
    packing=False,
)

# Evaluate and Metrics
from evaluate import load
import numpy as np

metric = load("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # If logits provided, convert to predicted token IDs
    if pred_ids.ndim == 3:
        pred_ids = np.argmax(pred_ids, axis=-1)

    # Replace -100 in label IDs to tokenizer.pad_token_id (common in masked labels)
    labels_ids = np.where(labels_ids != -100, labels_ids, tokenizer.pad_token_id)

    # Decode predictions and labels properly
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = metric.compute(predictions=pred_str, references=labels_str)
    return {
        "rouge1": rouge_output["rouge1"],
        "rouge2": rouge_output["rouge2"],
        "rougeL": rouge_output["rougeL"],
    }


# Initialize the trainer again with the new configurations
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset.select(range(100)),  # a subset for evaluation
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

# Start training with evaluation
trainer.train()

# Additional inference examples for evaluation
example_prompts = [
    "### Human: What event drove the social aspirations of the European working class into the background?\n### Assistant:",
    "### Human: Why was the Communist League dissolved?\n### Assistant:",
    "### Human: Who were commissioned to prepare a practical programme for the Communist League?\n### Assistant:",
]

for prompt in example_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=100)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
50,0.208800,0.641323,0.479473,0.384476,0.469007
100,0.937400,0.320995,0.533232,0.463495,0.528204
150,0.321100,0.254932,0.536368,0.471327,0.529181
200,0.365900,0.209067,0.538843,0.487283,0.536597
250,0.385900,0.131173,0.548001,0.509061,0.545638
300,0.187300,0.127852,0.553406,0.516154,0.551443
350,0.199500,0.097887,0.560225,0.524304,0.558551
400,0.112800,0.093327,0.553693,0.515495,0.550792
450,0.119500,0.084795,0.572110,0.535011,0.569465
500,0.121100,0.064762,0.578259,0.546930,0.577551


### Human: What event drove the social aspirations of the European working class into the background?
### Assistant: The defeat of the Parisian insurrection of June 1848 drove the aspirations of the working class into the background.

### Human: Why was the Communist League dissolved?
### Assistant: The remaining members dissolved the League after the defeat of the Paris Commune.

### Human: Who were commissioned to prepare a practical programme for the Communist League?
### Assistant: Marx and Engels were commissioned to prepare the programme for the Communist League.



In [12]:
!zip -r marx_finetuned_model.zip marx_finetuned_model

  adding: marx_finetuned_model/ (stored 0%)
  adding: marx_finetuned_model/checkpoint-500/ (stored 0%)
  adding: marx_finetuned_model/checkpoint-500/vocab.json (deflated 59%)
  adding: marx_finetuned_model/checkpoint-500/generation_config.json (deflated 30%)
  adding: marx_finetuned_model/checkpoint-500/model.safetensors (deflated 7%)
  adding: marx_finetuned_model/checkpoint-500/tokenizer_config.json (deflated 62%)
  adding: marx_finetuned_model/checkpoint-500/trainer_state.json (deflated 71%)
  adding: marx_finetuned_model/checkpoint-500/training_args.bin (deflated 51%)
  adding: marx_finetuned_model/checkpoint-500/optimizer.pt (deflated 9%)
  adding: marx_finetuned_model/checkpoint-500/scheduler.pt (deflated 55%)
  adding: marx_finetuned_model/checkpoint-500/merges.txt (deflated 53%)
  adding: marx_finetuned_model/checkpoint-500/tokenizer.json (deflated 82%)
  adding: marx_finetuned_model/checkpoint-500/rng_state.pth (deflated 25%)
  adding: marx_finetuned_model/checkpoint-500/speci

In [ ]:
!zip -r marx_finetuned_model.zip marx_finetuned_model_v2